In [1]:
# drivers_df =pd.read_csv("cleaned_data/drivers.csv", nrows = 5000)

In [2]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [3]:
#Test1 - 7572167 - GMC
#Test2 - 7572279 - toyota
#Test3 - 7572212 - Hyundai
#Test4 - 7571367 - Tesla - 847?
#dr 1 - 6212210 - AVG 670 - FORD
#dr 2 - 6074990 - AVG -710 - 710 - most accurate 

input_id = 6074990  
dr_df =pd.read_csv("cleaned_data/drivers.csv")
veh_df =pd.read_csv("cleaned_data/vehicles.csv")

In [4]:
drivers_df = dr_df[dr_df['VEHICLE_ID'] == input_id]
drivers_df.head()

,DRIVER_ID,GENDER,CREDIT_SCORE,VEHICLE_ID,AGE
29,30,1,750,6074990,45
191071,191930,1,710,6074990,63
295258,296774,1,710,6074990,46
295270,296786,1,710,6074990,25
295366,296882,1,710,6074990,46


In [5]:
sample_count = len(drivers_df)
sample_count

14

In [6]:
#Extend samples
if sample_count > 500:
    vehicle_entered = veh_df[veh_df['VEHICLE_ID'] == input_id]
else:
    vehicle_entered = veh_df[veh_df['VEHICLE_ID'] == input_id] 
    vehicle_make = vehicle_entered['MAKE'].item()
    vehicle_subset = veh_df[veh_df['MAKE'] == vehicle_make]
    
vehicle_subset.head()

,VEHICLE_ID,YEAR,MAKE,MODEL
1774,6074714,2012,JEEP,COMPASS SPT
1775,6074715,2017,JEEP,COMPASS SPT
1776,6074716,2012,JEEP,COMPASS SPT
1777,6074717,2013,JEEP,COMPASS SPT
1778,6074718,2014,JEEP,COMPASS LTD


In [7]:
#Extend samples to MAKE
if sample_count > 500:
    vehicle_entered = veh_df[veh_df['VEHICLE_ID'] == input_id]
else:
    vehicle_entered = veh_df[veh_df['VEHICLE_ID'] == input_id] 
    vehicle_make = vehicle_entered['MAKE'].item()
    vehicle_subset = veh_df[veh_df['MAKE'] == vehicle_make]
    
vehicle_subset.head()

,VEHICLE_ID,YEAR,MAKE,MODEL
1774,6074714,2012,JEEP,COMPASS SPT
1775,6074715,2017,JEEP,COMPASS SPT
1776,6074716,2012,JEEP,COMPASS SPT
1777,6074717,2013,JEEP,COMPASS SPT
1778,6074718,2014,JEEP,COMPASS LTD


In [8]:
drivers_subset = dr_df.merge(vehicle_subset, left_on='VEHICLE_ID', right_on='VEHICLE_ID')
drivers_subset.head()

,DRIVER_ID,GENDER,CREDIT_SCORE,VEHICLE_ID,AGE,YEAR,MAKE,MODEL
0,30,1,750,6074990,45,2012,JEEP,COMPASS LTD
1,191930,1,710,6074990,63,2012,JEEP,COMPASS LTD
2,296774,1,710,6074990,46,2012,JEEP,COMPASS LTD
3,296786,1,710,6074990,25,2012,JEEP,COMPASS LTD
4,296882,1,710,6074990,46,2012,JEEP,COMPASS LTD


In [9]:
sample_count = len(drivers_subset)
print(sample_count)

26505


In [10]:
# what size is the best.. 0ver 10,000 wouldn't run - 5000?

In [11]:
#Extend samples if necessary again or reduce to first 10,000 to 500
# Filter outliers
drivers_subset = drivers_subset[drivers_subset['CREDIT_SCORE'] > 300]
drivers_subset = drivers_subset[drivers_subset['AGE'] > 18]                                
sample_count = len(drivers_subset)
print(sample_count)  
                                
if sample_count > 5000:   
    drivers_subset = drivers_subset.head(5000)
    drivers_df = drivers_subset.drop(columns =['YEAR', 'MAKE', 'MODEL'])    
elif sample_count > 500:
    drivers_df = drivers_subset.drop(columns =['YEAR', 'MAKE', 'MODEL'])
else:
    vehicle_entered = veh_df[veh_df['VEHICLE_ID'] == input_id] 
    vehicle_year = vehicle_entered['YEAR'].item()
    vehicle_subset = veh_df[veh_df['YEAR'] == vehicle_year]

sample_count = len(drivers_df)
print(sample_count)
drivers_df.head()

26054
5000


,DRIVER_ID,GENDER,CREDIT_SCORE,VEHICLE_ID,AGE
0,30,1,750,6074990,45
1,191930,1,710,6074990,63
2,296774,1,710,6074990,46
3,296786,1,710,6074990,25
4,296882,1,710,6074990,46


In [12]:
sample_count = len(drivers_subset)
print(sample_count)
if sample_count < 500:
    drivers_subset = dr_df.merge(vehicle_subset, left_on='VEHICLE_ID', right_on='VEHICLE_ID')
    drivers_subset.head()
else:
    print("PASS...")

5000
PASS...


In [13]:
if sample_count < 500:
    sample_count = len(drivers_subset)
    print(sample_count)
    drivers_df = drivers_subset.drop(columns =['YEAR', 'MAKE', 'MODEL'])
    print(len(drivers_df))
else:
    print("PASS...")


PASS...


In [14]:
#Fianlly drop driver ID for the ML model

drivers_df = drivers_df.drop(columns='DRIVER_ID')
drivers_df.head()

,GENDER,CREDIT_SCORE,VEHICLE_ID,AGE
0,1,750,6074990,45
1,1,710,6074990,63
2,1,710,6074990,46
3,1,710,6074990,25
4,1,710,6074990,46


In [15]:
# Remove creadit score outcome target from fetures data
y = drivers_df['CREDIT_SCORE']
X = drivers_df.drop(columns='CREDIT_SCORE')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [16]:
from sklearn.ensemble import RandomForestClassifier
RF_clf = RandomForestClassifier()
RF_model = RF_clf.fit(X_train, y_train)
RF_prediction = RF_clf.predict(X_test)

In [18]:
# Setup the first data as input
import numpy as np
test_data = np.array([1,input_id,27])
test_data = test_data.reshape(-1, 3)
test_data = test_data.astype(float)

In [19]:
RF_model.predict(test_data)[0]  # Perfect

824